In [ ]:
# Install and load libraries

In [1]:
%conda install -c conda-forge lightgbm 

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::tqdm==4.62.3=pyhd8ed1ab_0
  - conda-forge/noarch::black==21.11b1=pyhd8ed1ab_0
  - conda-forge/linux-64::conda-package-handling==1.7.3=py38h497a2fe_1
  - conda-forge/noarch::dask-core==2021.11.2=pyhd8ed1ab_0
  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/linux-64::pytest==6.2.5=py38h578d9bd_1
  - conda-forge/linux-64::watchdog==2.1.6=py38h578d9bd_1
  - conda-forge/linux-64::aiohttp==3.8.1=py38h497a2fe_0
  - conda-forge/linux-64::astropy==5.0=py38h6c62de6_0
  - conda-forge/linux-64::bokeh==2.4.2=py38h578d9bd_0
  - conda-forge/linux-64::distributed==2021.11.2=py38h578d9bd_0
  - conda-forge/noarch::flask==2.0.2=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.5.0=py38hf4fb855_0
  - conda-forge/noarch::nbformat==5.1.3=pyhd8ed1ab_0
  - conda-forge/noarch::pylint==2.12.1=pyhd8ed1ab_0
  - conda-forg

In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import glob
from lightgbm import LGBMRegressor
import random
from sklearn.model_selection import ParameterSampler
import scipy
import gc
from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm

In [2]:
np.random.seed(54321)
random.seed(54321)

# Read in Data and Prepare for Modeling

In [3]:
# Create an empty list to hold the dataframes of highways england data
total_df_list = list()

# Loop through the files, sorted in alphabetical order
# Read them into a df, make sure they are sorted by timestamp, and append to the list
for fname in sorted(glob.glob("Data/Unseen Sensor/Processed/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname) #, parse_dates=['timestamp'], index_col=['timestamp'])
    df = df.sort_values(by="timestamp")
    total_df_list.append(df)

Reading Data/Unseen Sensor/Processed/A19-9336-1_Northbound_2019_Processed.csv
Reading Data/Unseen Sensor/Processed/A66-9521-1_Westbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M40-7048-2_Southbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M62-2056A_Eastbound_Processed.csv


In [4]:
# Read in the start and end points csv, and subtract 1 to deal with index differences between R and python
start_end = pd.read_csv("unseen_sensor_start_end_points.csv")
start_end["start"] = start_end["start"] - 1
start_end["end"] = start_end["end"]

In [5]:
# Create an empty list to hold the subset data frames (those with only 12 weeks of data per highway)
subset_df_list = list()

In [6]:
# For each df in our original total df list
for idx, df in enumerate(total_df_list):
        
    # Filter the timeframe based on the start_end_points csv files
    subset_df = df.iloc[start_end.iloc[idx,1]:start_end.iloc[idx,2], ]\
    .reset_index(drop=True).reset_index(drop=False)\
    .rename(columns={"index":"rn"})
    
    # Create a new field called train_val_test to differentiate each set of data
    subset_df["train_val_test"] = np.where(subset_df["rn"]<(96*7*8),
                                           "train",
                                           np.where(subset_df["rn"]<(96*7*10),
                                                    "val",
                                                    "test"
                                                   )
                                       )
    
    # Append to list
    subset_df_list.append(subset_df)

In [7]:
# Create a list of df's with only fields we need

# Initialize empty list
model_df_list = list()

# For df in subset list
for df in subset_df_list:
       
    # Extract the timestamp, the volume, and the train_val_test assignment
    model_df = df[['timestamp', 'total_volume', "train_val_test"]]\
    .rename(columns={'timestamp':'start', 'total_volume':'target'})
    
    # Append this df to the new list
    model_df_list.append(model_df)

## Create Lag Emebedded Matrices for each TS

In [10]:
# Set our final lag value to be 840
lag_n = 840

In [11]:
# # Lag embed the data frames and save to a list
lag_embed_df_list = list()

for df in model_df_list:
    # For each df in our list
    for n in range(1, (lag_n+1)):
        # For each lag level, up to 960
        # Create a new column called target-n
        name = f"target-{n}"
        # Save the target shifted n values into this colume
        df[name] = df['target'].shift(n)
    # Append to list
    lag_embed_df_list.append(df)

In [12]:
# Split the lag embedded list into train, val, and test lists

# First, initialize empty lists for each train, val, and test
train_df_list = list()
val_df_list = list()
test_df_list = list()

for i in range(len(lag_embed_df_list)):
    # For each df in our list
    df = lag_embed_df_list[i].copy()

    # Add a ts_index of i+1 to join with clustering data from R
    df['ts_index'] = i + 1
    
    # Subset into train, val, and test df's based on the train_val_test_field
    train_df = df.query("train_val_test == 'train'").copy()
    val_df = df.query("train_val_test=='val'").copy()
    test_df = df.query("train_val_test=='test'").copy()
    
    # Append to appropriate lists
    train_df_list.append(train_df)
    val_df_list.append(val_df)
    test_df_list.append(test_df)

In [13]:
# Concat all dfs from the lists together to create one full train, val, and test df
train_df_full = pd.concat(train_df_list)
val_df_full = pd.concat(val_df_list)
test_df_full = pd.concat(test_df_list)

In [14]:
# Drop unneeded columns
train_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
val_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
test_df_full.drop(columns=['start', 'train_val_test'], inplace=True)

In [15]:
# Append the training and validation data together for later use
train_val_df_full = train_df_full.append(val_df_full)

In [16]:
# Delete unused variables to free up memory
del train_df_list
del val_df_list 
del test_df_list
del lag_embed_df_list
del model_df_list
del subset_df_list

In [17]:
# Force garbage collection to free up memory
gc.collect()

0

In [18]:
unseen_sen_clust = pd.read_csv("Results/Unseen Sensor/clust_assign.csv")

In [19]:
unseen_sen_clust.head()

,Unnamed: 0,rand,catch22,tsfeat,dtw
0,1,3,1,2,2
1,2,2,1,2,1
2,3,1,1,2,1
3,4,4,1,1,1


# Full Model

In [20]:
# Save model to file to use later
filename = 'Results/Global/LightGBM Default/Full/model'
mod = joblib.load(filename)

In [21]:
res = pd.read_csv('Results/Global/LightGBM Default/Full/residual.csv')

In [22]:
# Function to compute test preds
def compute_lgbm_test_preds(mod, data, lag_n):
    """Function which takes in: a model, test data, and the lag embedding to use, and returns a df of forecasts"""

    # Initialize an empty data frame to store preds
    pred_df = pd.DataFrame()
    
    # Loop through each individual time series index in the data set
    for ts_idx in data.ts_index.unique():
        # Create the X matrix for each one
        X = data.query("ts_index==@ts_idx").iloc[:,1:(lag_n+1)].copy()

        # Forecast for that X matrix
        preds = mod.predict(X)
        
        # Save the results to a temp data frame
        pred_df_sub = pd.DataFrame({"ts_index": ts_idx, "test_preds": preds})
        
        # Append to primary data frame
        pred_df = pred_df.append(pred_df_sub)
    
    # Return df of all preds with corresponding ts_index column
    return pred_df

In [23]:
unseen_test_preds = compute_lgbm_test_preds(mod, test_df_full, lag_n)

In [24]:
# Function to compute performance metrics on test data
def compute_lgbm_test_perf(preds, data):
    """Function which takes inputs: a data frame of test predictions, and a test data df,
    and which returns a data frame of model performance"""
    
    # Create an empty list to store model performance
    perf_ls = list()
    
    # For each time series index in our data set
    for ts_idx in data.ts_index.unique():
        # Get the target (actual) for that index
        y_sub = data.query("ts_index==@ts_idx").iloc[:,0]
        # Extract the corresponding forecasts
        preds_sub = preds.query("ts_index==@ts_idx").test_preds
        
        # Compute rmse, mae, and the mean of the true target value for those preds
        rmse_sub = mean_squared_error(y_sub, preds_sub, squared=False)
        mae_sub = mean_absolute_error(y_sub, preds_sub)
        mean_sub = np.mean(y_sub)
        
        # Save those metrics to a dictionary
        pred_dict = {"rmse": rmse_sub, "mae": mae_sub, "mean": mean_sub}
        
        # Append the dictionary to the list
        perf_ls.append(pred_dict)
        
    # Return a data frame of model performance created from the list of dictionaries
    return pd.DataFrame(perf_ls)

In [25]:
# Compute model perf metrics using above function
full_mod_test_perf = compute_lgbm_test_perf(unseen_test_preds, test_df_full)

# Compute scaled performance metrics in new columns
full_mod_test_perf['nrmse'] = full_mod_test_perf['rmse']/full_mod_test_perf['mean']
full_mod_test_perf['smae'] = full_mod_test_perf['mae']/full_mod_test_perf['mean']

In [26]:
# Print the means of model perf metrics
full_mod_test_perf.mean()

rmse      41.842156
mae       27.496543
mean     377.001860
nrmse      0.122099
smae       0.084261
dtype: float64

In [27]:
# Function to compute pred intervals with bootstrap method
def compute_lgbm_boostrap_int(preds, resid, n_boot):
    """Function which takes in a model's predictions and residuals, and a number of bootstrap resamples to use,
    and which outputs a df with pred intervals at 80% and 95%"""
    
    # Set seeds for reproducibility
    random.seed(54321)
    np.random.seed(54321)
    
    # Create empty columns in the pred df to store the PIs
    preds['lo_95'] = np.nan
    preds['hi_95'] = np.nan
    preds['lo_80'] = np.nan
    preds['hi_80'] = np.nan
    
    # For each row in the pred df
    for n in range(preds.shape[0]):
        # Sample with replacement n_boot times from the residuals
        resid_boot = np.random.choice(resid, size=n_boot, replace=True)
        # Extract the forecast value for that row
        pred_n = preds.iloc[n, :].test_preds
        # Add the residual vector to the forecast value
        pred_n_boot = resid_boot + pred_n
        
        # Compute quantiles of this residual+forecast vector
        percent_95_lo = np.percentile(pred_n_boot, 2.5)
        percent_95_hi = np.percentile(pred_n_boot, 97.5)
        
        percent_80_lo = np.percentile(pred_n_boot, 10)
        percent_80_hi = np.percentile(pred_n_boot, 90)
        
        # Save these quantiles to the appropriate df column
        preds.iloc[n, 2] = percent_95_lo
        preds.iloc[n, 3] = percent_95_hi
        preds.iloc[n, 4] = percent_80_lo
        preds.iloc[n, 5] = percent_80_hi
    
    # Return the updated preds data frame
    return preds

In [28]:
# Compute PIs with 1000 bootstrap samples
full_mod_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds, 
                                               res.residual.values, 
                                               1000)

In [32]:
# Add the true values into their own df column
full_mod_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [33]:
full_mod_boot_ints.head()

,ts_index,test_preds,lo_95,hi_95,lo_80,hi_80,actual
0,1,304.771605,248.226806,366.975469,278.707203,334.089492,263.0
1,1,251.952300,194.668422,314.033011,221.810907,280.595017,242.0
2,1,226.077711,165.181467,294.471624,195.338483,254.830760,234.0
3,1,212.428004,151.769654,277.168829,184.059189,243.460251,188.0
4,1,178.551122,113.161899,238.991371,145.019888,207.924307,170.0


In [34]:
# Create a function to compute the interval score
def interval_score(true_values, lower, upper, interval_range):
    """ Function which takes in the true values, the upper and lower bounds of PIs, and the PI level (e.g., 90%)
        and from these inputs, computes the interval score for each prediction
    """
    
    # Compute alpha from the interval range
    alpha = 1-interval_range
    
    # Save the upper, lower, and true_values as numpy arrays for computation purposes
    upper = np.array(upper)
    lower = np.array(lower)
    true_values = np.array(true_values)
    
    # Compute the lower component of the interval score - just a boolean for true below interval
    def lower_ind(true,low):
        if true<low:
            return 1
        else:
            return 0
        
    # Computer the upper component of the interval score - similar boolean for true above interval
    def upper_ind(true,up):
        if true>up:
            return 1
        else:
            return 0
        
    # Computer the actual score for each obsveration - formula here: https://epiforecasts.io/scoringutils/reference/interval_score.html
    scores = (upper-lower) + (2/alpha)*(lower-true_values)*(lower > true_values) + (2/alpha)*(true_values-upper)*(true_values > upper)
    
    # Return the scores array
    return scores

In [35]:
# Compute the 95% and 80% PI scores using the above function as new data frame columns
full_mod_boot_ints['int_95_score'] = interval_score(full_mod_boot_ints.actual, 
                                                    full_mod_boot_ints.lo_95,
                                                    full_mod_boot_ints.hi_95,
                                                    0.95)
                                                    
full_mod_boot_ints['int_80_score'] = interval_score(full_mod_boot_ints.actual, 
                                                    full_mod_boot_ints.lo_80,
                                                    full_mod_boot_ints.hi_80,
                                                    0.80)

In [37]:
# Print the means of the interval scores
full_mod_boot_ints.mean()

ts_index          2.500000
test_preds      377.426473
lo_95           313.142994
hi_95           443.922244
lo_80           347.312792
hi_80           408.423554
actual          377.001860
int_95_score    364.511836
int_80_score    174.582233
dtype: float64

In [121]:
full_mod_boot_ints_group = full_mod_boot_ints.groupby("ts_index").mean().reset_index()
full_mod_boot_ints_group['mean'] = full_mod_test_perf['mean'].values
full_mod_boot_ints_group['int_95_score_scaled'] = full_mod_boot_ints_group['int_95_score']/full_mod_boot_ints_group['mean']
full_mod_boot_ints_group['int_80_score_scaled'] = full_mod_boot_ints_group['int_80_score']/full_mod_boot_ints_group['mean']

In [122]:
full_mod_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.988728
int_80_score_scaled    0.511796
dtype: float64

# Random Clusters

In [57]:
unseen_sen_clust.rand

0    3
1    2
2    1
3    4
Name: rand, dtype: int64

In [58]:
rand_clust_mod3 = joblib.load("Results/Global/LightGBM Default/Random Cluster/model_3")
rand_clust_mod2 = joblib.load("Results/Global/LightGBM Default/Random Cluster/model_2")
rand_clust_mod1 = joblib.load("Results/Global/LightGBM Default/Random Cluster/model_1")
rand_clust_mod4 = joblib.load("Results/Global/LightGBM Default/Random Cluster/model_4")

In [59]:
rand_clust_res = pd.read_csv("Results/Global/LightGBM Default/Random Cluster/residual.csv")
rand_clust_res['residual'] = rand_clust_res['residual'].apply(eval)

res_rand_clust_3 = rand_clust_res.query("cluster==3")['residual'].values[0]
res_rand_clust_2 = rand_clust_res.query("cluster==2")['residual'].values[0]
res_rand_clust_1 = rand_clust_res.query("cluster==1")['residual'].values[0]
res_rand_clust_4 = rand_clust_res.query("cluster==4")['residual'].values[0]

In [60]:
unseen_test_preds_rand_clust3 = compute_lgbm_test_preds(rand_clust_mod3, 
                                                        test_df_full.query("ts_index==1"), 
                                                        lag_n)

unseen_test_preds_rand_clust2 = compute_lgbm_test_preds(rand_clust_mod2, 
                                                        test_df_full.query("ts_index==2"), 
                                                        lag_n)

unseen_test_preds_rand_clust1 = compute_lgbm_test_preds(rand_clust_mod1, 
                                                        test_df_full.query("ts_index==3"), 
                                                        lag_n)

unseen_test_preds_rand_clust4 = compute_lgbm_test_preds(rand_clust_mod4, 
                                                        test_df_full.query("ts_index==4"), 
                                                        lag_n)

In [61]:
# Compute model perf metrics using above function
rand_clust3_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust3, test_df_full.query("ts_index==1"))
rand_clust2_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust2, test_df_full.query("ts_index==2"))
rand_clust1_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust1, test_df_full.query("ts_index==3"))
rand_clust4_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust4, test_df_full.query("ts_index==4"))

In [62]:
rand_clust_test_perf = rand_clust3_test_perf.append(rand_clust2_test_perf).append(rand_clust1_test_perf).append(rand_clust4_test_perf)

# Compute scaled performance metrics in new columns
rand_clust_test_perf['nrmse'] = rand_clust_test_perf['rmse']/rand_clust_test_perf['mean']
rand_clust_test_perf['smae'] = rand_clust_test_perf['mae']/rand_clust_test_perf['mean']

In [63]:
rand_clust_test_perf.mean()

rmse      42.711446
mae       28.379861
mean     377.001860
nrmse      0.123999
smae       0.085846
dtype: float64

In [64]:
# Compute PIs with 1000 bootstrap samples
nboot = 1000

rand3_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust3,
                                            res_rand_clust_3, 
                                            nboot)

rand2_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust2,
                                            res_rand_clust_2, 
                                            nboot)

rand1_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust1,
                                            res_rand_clust_1, 
                                            nboot)

rand4_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust4,
                                            res_rand_clust_4, 
                                            nboot)

In [65]:
rand_boot_ints = rand3_boot_ints.append(rand2_boot_ints).append(rand1_boot_ints).append(rand4_boot_ints)

In [66]:
# Add the true values into their own df column
rand_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [67]:
# Compute the 95% and 80% PI scores using the above function as new data frame columns
rand_boot_ints['int_95_score'] = interval_score(rand_boot_ints.actual, 
                                                    rand_boot_ints.lo_95,
                                                    rand_boot_ints.hi_95,
                                                    0.95)
                                                    
rand_boot_ints['int_80_score'] = interval_score(rand_boot_ints.actual, 
                                                    rand_boot_ints.lo_80,
                                                    rand_boot_ints.hi_80,
                                                    0.80)

In [68]:
rand_boot_ints.mean()

ts_index          2.500000
test_preds      375.772744
lo_95           316.066282
hi_95           437.109855
lo_80           346.402494
hi_80           406.045257
actual          377.001860
int_95_score    374.365077
int_80_score    181.435404
dtype: float64

In [123]:
rand_boot_ints_group = rand_boot_ints.groupby("ts_index").mean().reset_index()
rand_boot_ints_group['mean'] = rand_clust_test_perf['mean'].values
rand_boot_ints_group['int_95_score_scaled'] = rand_boot_ints_group['int_95_score']/rand_boot_ints_group['mean']
rand_boot_ints_group['int_80_score_scaled'] = rand_boot_ints_group['int_80_score']/rand_boot_ints_group['mean']

In [124]:
rand_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    1.002656
int_80_score_scaled    0.526698
dtype: float64

# Highway System

In [69]:
eng_mod = joblib.load("Results/Global/LightGBM Default/Highway System/model_1")

In [70]:
highway_res = pd.read_csv("Results/Global/LightGBM Default/Highway System/residual.csv")
highway_res['residual'] = highway_res['residual'].apply(eval)

eng_res = highway_res.query("cluster==1")['residual'].values[0]

In [71]:
unseen_test_preds_highway = compute_lgbm_test_preds(eng_mod, 
                                                    test_df_full,
                                                    lag_n)

In [72]:
highway_test_perf = compute_lgbm_test_perf(unseen_test_preds_highway, test_df_full)


In [73]:
highway_test_perf['nrmse'] = highway_test_perf['rmse']/highway_test_perf['mean']
highway_test_perf['smae'] = highway_test_perf['mae']/highway_test_perf['mean']

In [74]:
highway_test_perf.mean()

rmse      42.347075
mae       27.764704
mean     377.001860
nrmse      0.123377
smae       0.085117
dtype: float64

In [75]:
highway_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_highway,
                                              eng_res,
                                              nboot)

In [76]:
highway_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [77]:
highway_boot_ints['int_95_score'] = interval_score(highway_boot_ints.actual, 
                                                    highway_boot_ints.lo_95,
                                                    highway_boot_ints.hi_95,
                                                    0.95)
                                                    
highway_boot_ints['int_80_score'] = interval_score(highway_boot_ints.actual, 
                                                    highway_boot_ints.lo_80,
                                                    highway_boot_ints.hi_80,
                                                    0.80)

In [78]:
highway_boot_ints.mean()

ts_index          2.500000
test_preds      377.305552
lo_95           304.068483
hi_95           454.183336
lo_80           341.587625
hi_80           414.452887
actual          377.001860
int_95_score    357.302657
int_80_score    174.832742
dtype: float64

In [125]:
highway_boot_ints_group = highway_boot_ints.groupby("ts_index").mean().reset_index()
highway_boot_ints_group['mean'] = highway_test_perf['mean'].values
highway_boot_ints_group['int_95_score_scaled'] = highway_boot_ints_group['int_95_score']/highway_boot_ints_group['mean']
highway_boot_ints_group['int_80_score_scaled'] = highway_boot_ints_group['int_80_score']/highway_boot_ints_group['mean']

In [126]:
highway_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    1.039300
int_80_score_scaled    0.539604
dtype: float64

# Catch22 KMeans Clusters

In [79]:
unseen_sen_clust.catch22

0    1
1    1
2    1
3    1
Name: catch22, dtype: int64

In [80]:
catch22_mod1 = joblib.load("Results/Global/LightGBM Default/Catch22 KMeans/model_1")

In [81]:
catch22_res = pd.read_csv("Results/Global/LightGBM Default/Catch22 KMeans/residual.csv")
catch22_res['residual'] = catch22_res['residual'].apply(eval)

catch22_res1 = catch22_res.query("cluster==1")['residual'].values[0]

In [82]:
unseen_test_preds_catch22 = compute_lgbm_test_preds(catch22_mod1, 
                                                    test_df_full,
                                                    lag_n)

In [83]:
catch22_test_perf = compute_lgbm_test_perf(unseen_test_preds_catch22, test_df_full)


In [84]:
catch22_test_perf['nrmse'] = catch22_test_perf['rmse']/catch22_test_perf['mean']
catch22_test_perf['smae'] = catch22_test_perf['mae']/catch22_test_perf['mean']

In [85]:
catch22_test_perf.mean()

rmse      42.069581
mae       27.545796
mean     377.001860
nrmse      0.122236
smae       0.084184
dtype: float64

In [86]:
catch22_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_catch22,
                                               catch22_res1,
                                               1000)

In [87]:
catch22_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [88]:
catch22_boot_ints['int_95_score'] = interval_score(catch22_boot_ints.actual, 
                                                    catch22_boot_ints.lo_95,
                                                    catch22_boot_ints.hi_95,
                                                    0.95)
                                                    
catch22_boot_ints['int_80_score'] = interval_score(catch22_boot_ints.actual, 
                                                    catch22_boot_ints.lo_80,
                                                    catch22_boot_ints.hi_80,
                                                    0.80)

In [89]:
catch22_boot_ints.mean()

ts_index          2.500000
test_preds      377.360313
lo_95           312.974815
hi_95           444.316797
lo_80           345.486081
hi_80           409.939152
actual          377.001860
int_95_score    368.665484
int_80_score    174.748017
dtype: float64

In [127]:
catch22_boot_ints_group = catch22_boot_ints.groupby("ts_index").mean().reset_index()
catch22_boot_ints_group['mean'] = catch22_test_perf['mean'].values
catch22_boot_ints_group['int_95_score_scaled'] = catch22_boot_ints_group['int_95_score']/catch22_boot_ints_group['mean']
catch22_boot_ints_group['int_80_score_scaled'] = catch22_boot_ints_group['int_80_score']/catch22_boot_ints_group['mean']

In [128]:
catch22_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.992367
int_80_score_scaled    0.517228
dtype: float64

# Train and Test - TSFeat KMeans Clusters

In [90]:
unseen_sen_clust.tsfeat

0    2
1    2
2    2
3    1
Name: tsfeat, dtype: int64

In [91]:
tsfeat_mod2 = joblib.load("Results/Global/LightGBM Default/TSFeat KMeans/model_2")
tsfeat_mod1 = joblib.load("Results/Global/LightGBM Default/TSFeat KMeans/model_1")

In [92]:
tsfeat_res = pd.read_csv("Results/Global/LightGBM Default/TSFeat KMeans/residual.csv")
tsfeat_res['residual'] = tsfeat_res['residual'].apply(eval)

tsfeat_res2 = tsfeat_res.query("cluster==2")['residual'].values[0]
tsfeat_res1 = tsfeat_res.query("cluster==1")['residual'].values[0]

In [93]:
unseen_test_preds_tsfeat2 = compute_lgbm_test_preds(tsfeat_mod2, 
                                                    test_df_full.query("ts_index!=4"),
                                                    lag_n)

unseen_test_preds_tsfeat1 = compute_lgbm_test_preds(tsfeat_mod2, 
                                                    test_df_full.query("ts_index==4"),
                                                    lag_n)

In [96]:
tsfeat_test_perf2 = compute_lgbm_test_perf(unseen_test_preds_tsfeat2, test_df_full.query("ts_index!=4"))

tsfeat_test_perf1 = compute_lgbm_test_perf(unseen_test_preds_tsfeat1, test_df_full.query("ts_index==4"))

tsfeat_test_perf = tsfeat_test_perf2.append(tsfeat_test_perf1)

In [97]:
tsfeat_test_perf['nrmse'] = tsfeat_test_perf['rmse']/tsfeat_test_perf['mean']
tsfeat_test_perf['smae'] = tsfeat_test_perf['mae']/tsfeat_test_perf['mean']

In [99]:
tsfeat_test_perf

,rmse,mae,mean,nrmse,smae
0,22.009990,15.749714,199.294643,0.110439,0.079027
1,14.501915,10.338877,87.426339,0.165876,0.118258
2,40.833204,28.838692,405.072917,0.100805,0.071194
0,93.057442,56.384334,816.213542,0.114011,0.069080


In [98]:
tsfeat_test_perf.mean()

rmse      42.600638
mae       27.827904
mean     377.001860
nrmse      0.122783
smae       0.084390
dtype: float64

In [100]:
tsfeat_boot_ints2 = compute_lgbm_boostrap_int(unseen_test_preds_tsfeat2,
                                              tsfeat_res2,
                                              nboot)

tsfeat_boot_ints1 = compute_lgbm_boostrap_int(unseen_test_preds_tsfeat1,
                                              tsfeat_res1,
                                              nboot)

tsfeat_boot_ints = tsfeat_boot_ints2.append(tsfeat_boot_ints1)

In [101]:
tsfeat_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [102]:
tsfeat_boot_ints['int_95_score'] = interval_score(tsfeat_boot_ints.actual, 
                                                    tsfeat_boot_ints.lo_95,
                                                    tsfeat_boot_ints.hi_95,
                                                    0.95)
                                                    
tsfeat_boot_ints['int_80_score'] = interval_score(tsfeat_boot_ints.actual, 
                                                    tsfeat_boot_ints.lo_80,
                                                    tsfeat_boot_ints.hi_80,
                                                    0.80)

In [107]:
tsfeat_boot_ints.groupby('ts_index').agg({'int_80_score': 'mean', 'int_95_score': 'mean'})

,int_80_score,int_95_score
ts_index,,
1,83.708671,133.058487
2,66.585166,127.343683
3,158.863413,245.127729
4,397.829813,957.555793


In [105]:
tsfeat_boot_ints.mean()

ts_index          2.500000
test_preds      376.985258
lo_95           313.032813
hi_95           442.836050
lo_80           346.189179
hi_80           408.672506
actual          377.001860
int_95_score    365.771423
int_80_score    176.746766
dtype: float64

In [129]:
tsfeat_boot_ints_group = tsfeat_boot_ints.groupby("ts_index").mean().reset_index()
tsfeat_boot_ints_group['mean'] = tsfeat_test_perf['mean'].values
tsfeat_boot_ints_group['int_95_score_scaled'] = tsfeat_boot_ints_group['int_95_score']/tsfeat_boot_ints_group['mean']
tsfeat_boot_ints_group['int_80_score_scaled'] = tsfeat_boot_ints_group['int_80_score']/tsfeat_boot_ints_group['mean']

In [130]:
tsfeat_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.975636
int_80_score_scaled    0.515308
dtype: float64

# Train and Test - DTW Clusters

In [108]:
unseen_sen_clust.dtw

0    2
1    1
2    1
3    1
Name: dtw, dtype: int64

In [109]:
dtw_mod2 = joblib.load("Results/Global/LightGBM Default/DTW/model_2")
dtw_mod1 = joblib.load("Results/Global/LightGBM Default/DTW/model_1")

In [110]:
dtw_res = pd.read_csv("Results/Global/LightGBM Default/DTW/residual.csv")
dtw_res['residual'] = dtw_res['residual'].apply(eval)

dtw_res2 = dtw_res.query("cluster==2")['residual'].values[0]
dtw_res1 = dtw_res.query("cluster==1")['residual'].values[0]

In [111]:
unseen_test_preds_dtw2 = compute_lgbm_test_preds(dtw_mod2, 
                                                 test_df_full.query("ts_index==1"),
                                                 lag_n)

unseen_test_preds_dtw1 = compute_lgbm_test_preds(dtw_mod1, 
                                                 test_df_full.query("ts_index!=1"),
                                                 lag_n)

In [112]:
dtw_test_perf2 = compute_lgbm_test_perf(unseen_test_preds_dtw2, test_df_full.query("ts_index==1"))

dtw_test_perf1 = compute_lgbm_test_perf(unseen_test_preds_dtw1, test_df_full.query("ts_index!=1"))

dtw_test_perf = dtw_test_perf2.append(dtw_test_perf1)

In [113]:
dtw_test_perf['nrmse'] = dtw_test_perf['rmse']/dtw_test_perf['mean']
dtw_test_perf['smae'] = dtw_test_perf['mae']/dtw_test_perf['mean']

In [114]:
dtw_test_perf.mean()

rmse      43.121699
mae       28.311100
mean     377.001860
nrmse      0.137038
smae       0.094552
dtype: float64

In [115]:
dtw_boot_ints2 = compute_lgbm_boostrap_int(unseen_test_preds_dtw2,
                                           dtw_res2,
                                           nboot)

dtw_boot_ints1 = compute_lgbm_boostrap_int(unseen_test_preds_dtw1,
                                           dtw_res1,
                                           nboot)

In [116]:
dtw_boot_ints = dtw_boot_ints2.append(dtw_boot_ints1)
dtw_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [117]:
dtw_boot_ints['int_95_score'] = interval_score(dtw_boot_ints.actual, 
                                                    dtw_boot_ints.lo_95,
                                                    dtw_boot_ints.hi_95,
                                                    0.95)
                                                    
dtw_boot_ints['int_80_score'] = interval_score(dtw_boot_ints.actual, 
                                                    dtw_boot_ints.lo_80,
                                                    dtw_boot_ints.hi_80,
                                                    0.80)

In [118]:
dtw_boot_ints.mean()

ts_index          2.500000
test_preds      378.016010
lo_95           299.665418
hi_95           459.672752
lo_80           336.996847
hi_80           420.484967
actual          377.001860
int_95_score    327.764797
int_80_score    170.327615
dtype: float64

In [131]:
dtw_boot_ints_group = dtw_boot_ints.groupby("ts_index").mean().reset_index()
dtw_boot_ints_group['mean'] = dtw_test_perf['mean'].values
dtw_boot_ints_group['int_95_score_scaled'] = dtw_boot_ints_group['int_95_score']/dtw_boot_ints_group['mean']
dtw_boot_ints_group['int_80_score_scaled'] = dtw_boot_ints_group['int_80_score']/dtw_boot_ints_group['mean']

In [132]:
dtw_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    1.057273
int_80_score_scaled    0.589595
dtype: float64